# Pre-processing 

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!pip install emoji

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

In [ ]:
import json
import pandas as pd
import regex as re
import emoji

path = '/content/gdrive/MyDrive/IR_Project/Project/data1000/labeled/negative/timeline/'

from pathlib import Path
folder = Path(path).rglob('*.json')
files = [x for x in folder]

def get_emojis(s):
  orig_list = [s]
  emojis_iter = map(lambda y: y, emoji.UNICODE_EMOJI['en'].keys())
  regex_set = re.compile('|'.join(re.escape(em) for em in emojis_iter))
  new_list = regex_set.findall(orig_list[0])
  return new_list

pos_wrds = ['happy','pretty','good','joy','love','pride','win','certainty']
neg_wrds = ['hate','worthless','enemy','nervous','sad','afraid','tense','kill','grief','cry']
first_sing = ['i','me','my']
first_plu = [ 'we','u','our']

def pos_wrd_cnt(txt):
  count = 0
  for wrd in pos_wrds:
    patterns = ['^'+wrd,' '+ wrd+' ', wrd+'$']
    for pattern in patterns:
     count = count + len(re.findall(pattern, txt))
  return count
  

def neg_wrd_cnt(txt):
  count = 0
  for wrd in neg_wrds:
    patterns = ['^'+wrd,' '+ wrd+' ', wrd+'$']
    for pattern in patterns:
     count = count + len(re.findall(pattern, txt))
  return count

def first_sing_cnt(txt):
  count = 0
  for wrd in first_sing:
    patterns = ['^'+wrd,' '+ wrd+' ', wrd+'$']
    for pattern in patterns:
     count = count + len(re.findall(pattern, txt))
  return count

def first_plu_cnt(txt):
  count = 0
  for wrd in first_plu:
    patterns = ['^'+wrd,' '+ wrd+' ', wrd+'$']
    for pattern in patterns:
     count = count + len(re.findall(pattern, txt))
  return count

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

def lwr_spell(txt):
  ltxt = txt.lower()
  textBlb = TextBlob(ltxt)            # Making our first textblob
  textCorrected = textBlb.correct()   # Correcting the text
  return textCorrected

def tag_rem(txt,substr):
  txt = txt.replace('@' + substr , ' ') 
  return txt

def get_text(s):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', s)
lemoji = []
ltweets = []
lemojinum = []
lposemo = []
lnegemo = []
luserid = []
lsing = []
lplu = []

for file in files:
  data = pd.read_json(file,lines=True)
  
  if(data.empty):
    print("ERROR")
    continue
  date=data['created_at']
  length = len(date)
  text=data['text']
  
  user = data['user']
  entities=data['entities']
  
  tottext=''
  totemocnt = 0
  totposcnt = 0 
  totnegcnt = 0
  totfps = 0
  totfpp = 0
  totemo = []
  userid = user[0].get('id')
  
  
  for i in range(length):
    tagrem_txt = text[i].lower()
    if(len(entities[i].get('user_mentions'))!=0):
      en_user = entities[i].get('user_mentions')
      en_len = len( en_user)
      for k in range(en_len):
        tag = en_user[k].get('screen_name')
        tagrem_txt = tag_rem(tagrem_txt,tag.lower())

    txt_s= re.sub(r'^u ', 'you ', tagrem_txt) 
    txt_m = re.sub(r' u ', ' you ', txt_s)
    txt_e = re.sub(r' u$', ' you', txt_m)
    emojis = get_emojis(text[i])
    totemo = totemo + emojis
    text_rt= re.sub(r'^rt ', '', txt_e)    
    linkrem_txt= re.sub(r'https?:\/\/.*[\r\n]*', '', text_rt, flags=re.MULTILINE)
    puncrem_txt = re.sub(r'[^\w\s]', '', linkrem_txt) 

    lem_txt = lemmatize_words(puncrem_txt)
    tweet = get_text(lem_txt)
    
    tottext = tottext +tweet+'*'
    if((i+1)%20==0):
      lemoji.append(totemo)
      lposemo.append(pos_wrd_cnt(tottext))
      lnegemo.append(neg_wrd_cnt(tottext))
      lemojinum.append(len(totemo))
      lsing.append(first_sing_cnt(tottext))
      lplu.append(first_plu_cnt(tottext))
         
      ltweets.append(tottext)
      luserid.append(userid)
      tottext=''
      totemo = []

  lemoji.append(totemo)
  lposemo.append(pos_wrd_cnt(tottext))
  lnegemo.append(neg_wrd_cnt(tottext))
  lemojinum.append(len(totemo))
  lsing.append(first_sing_cnt(tottext))
  lplu.append(first_plu_cnt(tottext))
  #print(tottext)
  
  ltweets.append(tottext)
  luserid.append(userid)
   

dict = {'userid':luserid,'tweet': ltweets, 'emoji': lemoji, 'num_emoji': lemojinum,'+ve_emo_wrdcount':lposemo, 
          '-ve_emo_wrdcount': lnegemo, 'first_person_sing':lsing,'first_person_plu':lplu}  
  
df = pd.DataFrame(dict) 
df.to_csv(r'/content/gdrive/MyDrive/IR_Project/Project/data1000/labeled/negative/features_negative_20.csv',  index=False) 